In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# utilizado para la manipulación de directorios y rutas
import os

# Cálculo científico y vectorial para python
import numpy as np

# Libreria para graficos
from matplotlib import pyplot

# Modulo de optimizacion en scipy
from scipy import optimize

# modulo para cargar archivos en formato MATLAB
# from scipy.io import loadmat

# le dice a matplotlib que incruste gráficos en el cuaderno
%matplotlib inline

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

# Leer el archivo CSV utilizando el delimitador adecuado (;)
df = pd.read_csv('/content/drive/MyDrive/primelParcial/data2/hotelprepa2.csv', delimiter=';')
num_labels = 2
# Manejar valores faltantes si los hay
df.dropna(inplace=True)  # Eliminar filas con valores faltantes, ajusta según tu caso

# Codificar las variables categóricas
label_encoders = {}
for column in df.select_dtypes(include='object').columns:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le

# Cambiar el valor de la variable objetivo 'hotel'
df['hotel'] = df['hotel'].replace({'City Hotel': 1, 'Resort Hotel': 0})

# Separar las características (features) de la variable objetivo (target)
X = df.drop('hotel', axis=1)
y = df['hotel']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Dimensiones de X_train:", X_train.shape)
print("Dimensiones de X_test:", X_test.shape)
print("Dimensiones de y_train:", y_train.shape)
print("Dimensiones de y_test:", y_test.shape)



Dimensiones de X_train: (95118, 27)
Dimensiones de X_test: (23780, 27)
Dimensiones de y_train: (95118,)
Dimensiones de y_test: (23780,)


In [ ]:
import numpy as np

def normalizarCaracteristicas(X_train, X_test):
    mu_train = np.mean(X_train, axis=0)
    sigma_train = np.std(X_train, axis=0)

    X_train_norm = (X_train - mu_train) / sigma_train
    X_test_norm = (X_test - mu_train) / sigma_train

    return X_train_norm, mu_train, sigma_train, X_test_norm

# Llamada a la función para normalizar características
X_train_norm, mu_train, sigma_train, X_test_norm = normalizarCaracteristicas(X_train, X_test)

# Mostrar las dimensiones de los conjuntos normalizados
print("Dimensiones de X_train_norm:", X_train_norm.shape)
print("Dimensiones de X_test_norm:", X_test_norm.shape)



Dimensiones de X_train_norm: (95118, 27)
Dimensiones de X_test_norm: (23780, 27)


In [ ]:
def normalizarCaracteristicas(X_train, X_test):
    mu_train = np.mean(X_train, axis=0)
    sigma_train = np.std(X_train, axis=0)

    X_train_norm = (X_train - mu_train) / sigma_train
    X_test_norm = (X_test - mu_train) / sigma_train

    return X_train_norm, mu_train, sigma_train, X_test_norm

In [ ]:
# usare la sigmoide ´para  tomar un valor real como entrada y produce un valor en el rango de 0 a 1 como salida
def sigmoid(z):

    return 1.0 / (1.0 + np.exp(-z))

In [ ]:
#Usare una función de costo y su gradiente para un problema de regresión logística regularizada.
#para encontrar los parámetros theta que minimizan la función de costo.

def lrCostFunction(theta, X, y, lambda_):

    m = y.size

    # convierte las etiquetas a valores enteros si son boleanos
    if y.dtype == bool:
        y = y.astype(int)

    J = 0
    grad = np.zeros(theta.shape)

    h = sigmoid(X.dot(theta.T))

    temp = theta
    temp[0] = 0

#     J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h)))
    J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))

    grad = (1 / m) * (h - y).dot(X)
#     theta = theta - (alpha / m) * (h - y).dot(X)
    grad = grad + (lambda_ / m) * temp

    return J, grad
#    return J, theta

In [ ]:
# un ejemplo de calculo del costo
theta_initial = np.zeros(X_train_norm.shape[1])

# Definir el valor de lambda
lambda_ = 0.1

# Calcular el costo y el gradiente utilizando la función de costo
costo, _ = lrCostFunction(theta_initial, X_train_norm, y_train, lambda_)

print("Costo de entrenamiento:", costo)


Costo de entrenamiento: 0.6931471805599182


In [ ]:
#usare scipy  para utilizar sus funcionalidades de optimización numérica en Python.
#entrenare con  "One-vs-All",para que me devuelba loss parámetros óptimos del modelo para cada clase.
from scipy import optimize

def oneVsAll(X, y, num_labels, lambda_):
    # Algunas variables útiles
    m, n = X.shape

    # Inicializar matriz de parámetros theta
    all_theta = np.zeros((num_labels, n + 1))

    # Agregar una columna de unos a la matriz X
    X = np.concatenate([np.ones((m, 1)), X], axis=1)

    # Optimizar los parámetros theta para cada clase
    for c in range(num_labels):
        initial_theta = np.zeros(n + 1)
        options = {'maxiter': 50}  # Opciones para el optimizador
        # Minimizar la función de costo regularizada utilizando el método de conjugado de gradiente
        res = optimize.minimize(lrCostFunction, initial_theta, (X, (y == c), lambda_), jac=True, method='CG', options=options)
        # Asignar los parámetros theta óptimos a la fila correspondiente en all_theta
        all_theta[c] = res.x

    return all_theta


In [ ]:
#entreno un clasificador usando uno vs todos e imprimo mis valores de mis clases
num_labels = 2
lambda_ = 0.1
all_theta = oneVsAll(X, y, num_labels, lambda_)
print(all_theta.shape)

<ipython-input-34-0fae340bc49a>:21: RuntimeWarning: divide by zero encountered in log
  J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))
<ipython-input-34-0fae340bc49a>:21: RuntimeWarning: divide by zero encountered in log
  J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))
<ipython-input-34-0fae340bc49a>:21: RuntimeWarning: divide by zero encountered in log
  J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))
<ipython-input-34-0fae340bc49a>:21: RuntimeWarning: divide by zero encountered in log
  J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))
<ipython-input-34-0fae340bc49a>:21: RuntimeWarning: divide by zero encountered in log
  J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) *

(2, 28)


In [ ]:
#tomare los valores óptimos del modelo (all_theta) y un conjunto de datos de entrada (X), y
#predice las etiquetas de clase para cada muestra en X utilizando el enfoque de "One-vs-All" en la regresión logística

def predictOneVsAll(all_theta, X):

    m = X.shape[0];
    num_labels = all_theta.shape[0]

    p = np.zeros(m)

    # Add ones to the X data matrix
    X = np.concatenate([np.ones((m, 1)), X], axis=1)
    p = np.argmax(sigmoid(X.dot(all_theta.T)), axis = 1)

    return p

In [ ]:
# Usando mi 20% de datos de prueba
print(X_test_norm.shape)

# realizar predicciones en el conjunto de prueba usando el 20%
pred_test = predictOneVsAll(all_theta, X_test_norm)

# calcular la precisión en el conjunto de prueba usando mi 20% de y de prueba
accuracy_test = np.mean(pred_test == y_test) * 100

# Imprimir la precisión del conjunto de prueba
print('Precisión en el conjunto de prueba: {:.2f}%'.format(accuracy_test))

# seleccionar un subconjunto de datos de prueba para mostrar un ejemplo
X_prueba_subconjunto = X_test_norm.iloc[10:150, :]  # Utilizar .iloc para acceder a las filas
y_prueba_subconjunto = y_test.iloc[10:150]

# ahora agregar una columna de unos a la matriz X_prueba_subconjunto para realizar mis predicciones
X_prueba_subconjunto = np.concatenate([np.ones((X_prueba_subconjunto.shape[0], 1)), X_prueba_subconjunto], axis=1)

# realizar las predicciones en el subconjunto de datos de prueba
pred_prueba_subconjunto = np.argmax(sigmoid(X_prueba_subconjunto.dot(all_theta.T)), axis=1)

# Imprimir las predicciones y las etiquetas del mi 20% de datos de prueba
print("Predicciones del subconjunto de datos de prueba:")
print(pred_prueba_subconjunto)
print("Etiquetas reales del subconjunto de datos de prueba:")
print(y_prueba_subconjunto)


(23780, 27)
Precisión en el conjunto de prueba: 67.69%
Predicciones del subconjunto de datos de prueba:
[0 1 0 1 0 0 1 0 0 0 0 1 1 1 1 0 1 1 1 1 1 1 0 0 1 0 0 0 0 1 1 0 0 0 1 0 0
 1 1 0 0 0 1 1 1 1 0 0 1 0 0 1 0 1 1 0 0 0 1 0 1 0 0 1 1 0 0 1 0 0 1 1 0 1
 0 0 1 0 0 1 0 1 0 1 1 1 0 0 0 1 0 1 0 1 1 1 0 1 1 0 0 1 0 0 0 0 0 1 0 1 0
 0 1 0 0 0 1 0 0 1 0 0 1 1 0 1 1 1 1 1 1 0 0 0 0 1 1 1 1 0]
Etiquetas reales del subconjunto de datos de prueba:
60845     0
24618     1
86918     0
96103     0
115636    0
         ..
38293     1
28319     1
61462     0
15933     1
71819     0
Name: hotel, Length: 140, dtype: int64
